# Web Scraping: BeautifulSoup
_Collecting data from the internet and parsing it into meaningful (often tabular) form._

### Docs

- [BeautifulSoup documentation](https://www.crummy.com/software/BeautifulSoup/bs4/doc/)

### Installation

If not using the Metis kernel, please install the following libraries

With conda:
- `conda install beautifulsoup4 requests lxml html5lib`

With pip:
- `pip install beautifulsoup4 requests lxml html5lib`

If you have installed everything correctly, you should now be able to import these libraries. (You may need to shutdown and restart this notebook for BeautifulSoup to recognize the `lxml` and `html5lib` parsers.)

In [2]:
from bs4 import BeautifulSoup
import requests

## What is BeautifulSoup?

- Python library
- **HTML parser**:  Interprets structure of HTML file 
- Does not actually get pages from the web.  Use `requests` library for that.

<br>
<img src="images/web_scraping_pipeline.png" alt="Web Scraping Pipeline" style="width: 650px;"/>

## Intro to HTML

_Basic language used to create a webpage._

- Tells browser what, where, and how to display text, images, and other media
- Structured, hierarchical nature 
- Comprised of "elements" with properties

Example HTML element
```html
<tag-name attr1="value of attr1" attr2="value of attr2" .... attrN="value of attrN">
    Inner text of the tag
</tag-name>
```

### Tags,  `<tag-name>`

- Elements are labeled with tags
- Tells us what type of "thing" to render

    
| Tag | Use
|---          | ---|
|`h1`, `h2`, ..., `h6`| headers|
|`p`| paragraphs |
|`a`| anchors (e.g. links) |
|`div`| divisions (or sections) of a page |
|`img`| images |
|`li` | list items |


### Attributes, `attr1`

- Special properties we want this tag to have
- Typically appear as `attribute name = "attribute value"` pair

| Attribute | Use | Notes
|---   | ---       | ---|
|`href` | hyperlink reference | Clicking this element directs user to value url|
|`class`| style class | Many elements may have same class |
|`id`| unique identifier | Only one element per id! |
|`style`| extra element styling | Bad practice, use css instead |

### Inner HTML Text

- Text that appears between tags
- Often the information we want to extract during web scraping

### HTML Structure

A full HTML document has a structure similar to this:

```html
<html> 
  <head> </head>
  <body>
     <h1>This is a header</h1>
     <p style="color:red;" id="learning_paragraph">You are learning HTML</p>
     <a href="www.google.com">A link to Google</a>
  </body>
</html>
```

**QUESTIONS**
> How many elements do we have within the HTML body?  What are their tags?

> What is the inner HTML of the header element?

> What attribute(s) does the paragraph have?  And the attribute value(s)?

Saving this code as a .html file and opening it in a browser should yield:

<br>
<img src="images/example_html.png" alt="Rendering of Example HTML" style="width: 300px;" align="left"/>

## Learn to Scrape with Dummy HTML

Let's begin learning how to scrape by working with some dummy HTML, written below as a string.

In [4]:
my_html = """
<html>

<head>
</head>

<body>
    <div style="border: 1px solid">
        There isn't much in this file, except a list of to-do items. 
        <ul>
          <li>Make coffee</li>
          <li>Sweep the floor</li>
          <li>Go to the store</li>
          <li>Write BeautifulSoup lecture</li>
        </ul>
    </div>
</body>

</html>
"""

Let's take a look at this simple webpage.

In [5]:
from IPython.core.display import display, HTML
display(HTML(my_html))     # make sure Jupyter knows to display it as HTML

If we want to grab the four items on our to-do list and analyze them, we can use Beautiful Soup!

In [6]:
soup = BeautifulSoup(my_html, "html5lib")

Simply looking at `soup` isn't very useful -- it's just our HTML repeated back to us.

In [7]:
soup

<html><head>
</head>

<body>
    <div style="border: 1px solid">
        There isn't much in this file, except a list of to-do items. 
        <ul>
          <li>Make coffee</li>
          <li>Sweep the floor</li>
          <li>Go to the store</li>
          <li>Write BeautifulSoup lecture</li>
        </ul>
    </div>



</body></html>

### `.find()`

But Beautiful Soup also knows how to navigate this HTML.  We can use the `find` command to get to a specific element.

In [8]:
soup.find('li')  #Grabs the first element tagged as li

<li>Make coffee</li>

In [9]:
type(soup.find('li'))

bs4.element.Tag

`find` returns a tagged element, but we can go further and just select this element's inner HTML text.

In [10]:
soup.find('li').text

'Make coffee'

In [11]:
type(soup.find('li').text)

str

### `.find_all()`

Instead of selecting just one of our list items, we can get all of them by using `find_all`.  

This method looks for all instances matching our criteria on the entire HTML and gives us back a list.

In [12]:
soup.find_all('li')

[<li>Make coffee</li>,
 <li>Sweep the floor</li>,
 <li>Go to the store</li>,
 <li>Write BeautifulSoup lecture</li>]

To analyze our to-do list, we probably just want the text from each tagged element.  How can we do that?

One approach is to loop through the list and apply `.text` to each element:

In [13]:
todos=[]

for element in soup.find_all('li'):
    todos.append(element.text)
    
print(todos)

['Make coffee', 'Sweep the floor', 'Go to the store', 'Write BeautifulSoup lecture']


Or we could use a list comprehension:

In [14]:
todos=[element.text for element in soup.find_all('li')]

todos

['Make coffee',
 'Sweep the floor',
 'Go to the store',
 'Write BeautifulSoup lecture']

Now we have a clean list of strings, ready for analysis!

## Scrape Select Items on a Test Webpage

Now on to a more complicated example.  Take a look at `test_webpage/page.html`.  Let's try to grab all of the article links, like the ones for Starbucks and Bitcoin.

First get the HTML and then parse it with BeautifulSoup.

In [20]:
#webpage_string = 'https://www.imdb.com/name/nm0001202/'
webpage_string = 'https://www.imdb.com/name/nm0001202?ref_=nv_sr_srsg_0'

In [21]:
#webpage_string = 'test_webpage/page.html'

webpage_string = 'https://www.imdb.com/name/nm0001202?ref_=nv_sr_srsg_0'
with open(webpage_string) as page:
    test_html = page.read()
#soup = BeautifulSoup(test_html, 'lxml')    
soup = BeautifulSoup(test_html)

FileNotFoundError: [Errno 2] No such file or directory: 'https://www.imdb.com/name/nm0001202?ref_=nv_sr_srsg_0'

Links show up as `a` tags.  Let's just try to grab all of them.

In [14]:
soup.find_all('a')

[<a href="sports.html">Sports</a>,
 <a href="entertainment.html">Entertainment</a>,
 <a href="technology.html">Technology</a>,
 <a href="http://starbucks.com/drinks/unicorn.html">unicorn frappaccino</a>,
 <a href="https://finance.yahoo.com/quote/BTC-USD/">Bitcoin</a>,
 <a href="https://en.wikipedia.org/wiki/Cryptocurrency">cryptocurrency</a>,
 <a href="negative.html">negative</a>,
 <a href="https://en.wikipedia.org/wiki/Fake_news">"fake news"</a>]

Uh oh!  Looks like there are some links on the sidebar and in the footer, too.  We want only the ones in the articles so we'll need a better strategy.

Digging into the source code, it turns out that each of the articles live within a `div` labeled with the class `article`.  Let's try to get those.

### `class_` and `id_` 
The `find` and `find_all` methods take optional attribute arguments so you can filter down to elements with specific attributes like classes and ids.

In [15]:
soup.find_all('div', class_='article')

[<div class="article">
 <div class="title">New drink released today</div>
 <div class="summary">
         Starbucks is releasing a new drink today called the <a href="http://starbucks.com/drinks/unicorn.html">unicorn frappaccino</a>.
       </div>
 </div>, <div class="article">
 <div class="title">Bitcoin crashes</div>
 <div class="summary">
 <a href="https://finance.yahoo.com/quote/BTC-USD/">Bitcoin</a> dropped 200% today becoming the first <a href="https://en.wikipedia.org/wiki/Cryptocurrency">cryptocurrency</a> with <a href="negative.html">negative</a> value!
       </div>
 </div>]

There are our articles!  

Each of these `div` elements are also soup objects, so we can now query these `div`s to drill down further to just the links.

In [16]:
for div in soup.find_all('div', class_='article'):
    for link in div.find_all('a'):
        print(link)

<a href="http://starbucks.com/drinks/unicorn.html">unicorn frappaccino</a>
<a href="https://finance.yahoo.com/quote/BTC-USD/">Bitcoin</a>
<a href="https://en.wikipedia.org/wiki/Cryptocurrency">cryptocurrency</a>
<a href="negative.html">negative</a>


Excellent!  What if we want to print out the link text and the url it points to?


### `.get()`
The `get` method allows you access to any attribute of the element.

In [17]:
for div in soup.find_all('div', class_='article'):
    for link in div.find_all('a'):
        print(f'{link.text:20s} ---> {link.get("href")}')

unicorn frappaccino  ---> http://starbucks.com/drinks/unicorn.html
Bitcoin              ---> https://finance.yahoo.com/quote/BTC-USD/
cryptocurrency       ---> https://en.wikipedia.org/wiki/Cryptocurrency
negative             ---> negative.html


## Scrape the Web

So far we've used BeautifulSoup to parse our own HTML strings and files.  Now let's scrape Box Office Mojo.

First let's take a look at some source code for _The Big Lebowski_.

- Navigate to https://www.boxofficemojo.com/title/tt0118715/ in your browser, preferably Chrome
- Right click and select "Inspect"
- Alternatively, you can "View Page Source"

To retrieve the HTML for this webpage, we will use `requests`.

### `requests`

The `requests` library allows us to grab information from the web.  There are two common types of requests:
- `get` -- simply requests information, akin to putting a url in your browser
- `post` -- sends information to the website, for example, writing an email

We will be using `get` to retrieve a page's HTML.

In [57]:
#url = 'https://www.boxofficemojo.com/title/tt0118715/' 
#url = 'https://www.imdb.com/name/nm0001202?ref_=nv_sr_srsg_0'
fassbinder_page = 'https://www.imdb.com/name/nm0001202/?ref_=nv_sr_srsg_0#director'
godard_page = 'https://www.imdb.com/name/nm0000419/?ref_=nv_sr_srsg_0'
director_page = godard_page
url = director_page
#url = 'https://imslp.org/'
response = requests.get(url)
#response = requests.get(webpage_string )

The response we got back is an object that gives us access to:
- `response.text` -- the returned HTML (if any)
- `response.json` -- the returned JSON (if any), typical for APIs
- `response.status_code` -- a [code](https://en.wikipedia.org/wiki/List_of_HTTP_status_codes) to tell you if your request was successful or if an error occurred, 2XX indicates success while 404 means not found

In [58]:
response.status_code  #200 = success!

200

In [59]:
response.text[:1000]  #First 1000 characters of the HTML

'\n\n\n<!DOCTYPE html>\n<html\n    xmlns:og="http://ogp.me/ns#"\n    xmlns:fb="http://www.facebook.com/2008/fbml">\n    <head>\n         \n        <meta charset="utf-8">\n        <meta http-equiv="X-UA-Compatible" content="IE=edge">\n\n    <meta name="apple-itunes-app" content="app-id=342792525, app-argument=imdb:///name/nm0000419?src=mdot">\n\n\n\n        <script type="text/javascript">var IMDbTimer={starttime: new Date().getTime(),pt:\'java\'};</script>\n\n<script>\n    if (typeof uet == \'function\') {\n      uet("bb", "LoadTitle", {wb: 1});\n    }\n</script>\n  <script>(function(t){ (t.events = t.events || {})["csm_head_pre_title"] = new Date().getTime(); })(IMDbTimer);</script>\n        <title>Jean-Luc Godard - IMDb</title>\n  <script>(function(t){ (t.events = t.events || {})["csm_head_post_title"] = new Date().getTime(); })(IMDbTimer);</script>\n<script>\n    if (typeof uet == \'function\') {\n      uet("be", "LoadTitle", {wb: 1});\n    }\n</script>\n<script>\n    if (typeof uex 

In [60]:
page = response.text

### `BeautifulSoup` Basics

Now that we have the HTML, let's learn its structure by parsing with BeautifulSoup.

In [61]:

soup = BeautifulSoup(page, "lxml")


In [62]:
print(soup)

<!DOCTYPE html>
<html xmlns:fb="http://www.facebook.com/2008/fbml" xmlns:og="http://ogp.me/ns#">
<head>
<meta charset="utf-8"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta content="app-id=342792525, app-argument=imdb:///name/nm0000419?src=mdot" name="apple-itunes-app"/>
<script type="text/javascript">var IMDbTimer={starttime: new Date().getTime(),pt:'java'};</script>
<script>
    if (typeof uet == 'function') {
      uet("bb", "LoadTitle", {wb: 1});
    }
</script>
<script>(function(t){ (t.events = t.events || {})["csm_head_pre_title"] = new Date().getTime(); })(IMDbTimer);</script>
<title>Jean-Luc Godard - IMDb</title>
<script>(function(t){ (t.events = t.events || {})["csm_head_post_title"] = new Date().getTime(); })(IMDbTimer);</script>
<script>
    if (typeof uet == 'function') {
      uet("be", "LoadTitle", {wb: 1});
    }
</script>
<script>
    if (typeof uex == 'function') {
      uex("ld", "LoadTitle", {wb: 1});
    }
</script>
<link href="https://www.imdb.com/n

The `prettify` method turns the soup into a nicely formatted Unicode string with one tag on each line for readability.

In [63]:
print(soup.prettify())

<!DOCTYPE html>
<html xmlns:fb="http://www.facebook.com/2008/fbml" xmlns:og="http://ogp.me/ns#">
 <head>
  <meta charset="utf-8"/>
  <meta content="IE=edge" http-equiv="X-UA-Compatible"/>
  <meta content="app-id=342792525, app-argument=imdb:///name/nm0000419?src=mdot" name="apple-itunes-app"/>
  <script type="text/javascript">
   var IMDbTimer={starttime: new Date().getTime(),pt:'java'};
  </script>
  <script>
   if (typeof uet == 'function') {
      uet("bb", "LoadTitle", {wb: 1});
    }
  </script>
  <script>
   (function(t){ (t.events = t.events || {})["csm_head_pre_title"] = new Date().getTime(); })(IMDbTimer);
  </script>
  <title>
   Jean-Luc Godard - IMDb
  </title>
  <script>
   (function(t){ (t.events = t.events || {})["csm_head_post_title"] = new Date().getTime(); })(IMDbTimer);
  </script>
  <script>
   if (typeof uet == 'function') {
      uet("be", "LoadTitle", {wb: 1});
    }
  </script>
  <script>
   if (typeof uex == 'function') {
      uex("ld", "LoadTitle", {wb: 1});


In [64]:
soup.find_all('div', class_='filmo-category-section')

[<div class="filmo-category-section">
 <div class="filmo-row odd" id="director-tt12089010">
 <span class="year_column">
  2018
 </span>
 <b><a href="/title/tt12089010/">Spot of the 22nd Ji.hlava IDFF</a></b>
 (Short)
 <br/>
 </div>
 <div class="filmo-row even" id="director-tt5749596">
 <span class="year_column">
  2018
 </span>
 <b><a href="/title/tt5749596/">The Image Book</a></b>
 <br/>
 </div>
 <div class="filmo-row odd" id="director-tt8362750">
 <span class="year_column">
  2018
 </span>
 <b><a href="/title/tt8362750/">Bande-annonce de 'Le livre d'image'</a></b>
 (Short)
 <br/>
 </div>
 <div class="filmo-row even" id="director-tt4529452">
 <span class="year_column">
  2015
 </span>
 <b><a href="/title/tt4529452/">Remerciements de Jean-Luc Godard à son Prix d'honneur du cinéma suisse</a></b>
 (Short)
 <br/>
 </div>
 <div class="filmo-row odd" id="director-tt3495794">
 <span class="year_column">
  2014
 </span>
 <b><a href="/title/tt3495794/">Ponts de Sarajevo</a></b>
 (segment "Le p

In [65]:
soup.find_all('div', class_='filmo-row odd')

[<div class="filmo-row odd" id="director-tt12089010">
 <span class="year_column">
  2018
 </span>
 <b><a href="/title/tt12089010/">Spot of the 22nd Ji.hlava IDFF</a></b>
 (Short)
 <br/>
 </div>,
 <div class="filmo-row odd" id="director-tt8362750">
 <span class="year_column">
  2018
 </span>
 <b><a href="/title/tt8362750/">Bande-annonce de 'Le livre d'image'</a></b>
 (Short)
 <br/>
 </div>,
 <div class="filmo-row odd" id="director-tt3495794">
 <span class="year_column">
  2014
 </span>
 <b><a href="/title/tt3495794/">Ponts de Sarajevo</a></b>
 (segment "Le pont des soupirs")
 <br/>
 </div>,
 <div class="filmo-row odd" id="director-tt3749888">
 <span class="year_column">
  2014
 </span>
 <b><a href="/title/tt3749888/">Khan Khanne</a></b>
 (Short)
 <br/>
 </div>,
 <div class="filmo-row odd" id="director-tt1438535">
 <span class="year_column">
  2010
 </span>
 <b><a href="/title/tt1438535/">Film socialisme</a></b>
 <br/>
 </div>,
 <div class="filmo-row odd" id="director-tt1350630">
 <span 

In [66]:
len(soup.find_all('div', class_='filmo-row odd') )


298

In [67]:
soup.find('div', id_='director-tt0082671')

In [68]:
filmography = soup.find('div', id='filmography')
print(filmography)
print(len(filmography.find_all('div')))

<div id="filmography">
<div class="head" data-category="director" id="filmo-head-director" onclick="toggleFilmoCategory(this);">
<span class="hide-link" id="hide-director">Hide <img alt="Hide" class="absmiddle" height="16" src="https://m.media-amazon.com/images/S/sash/5o6LtZOQM$gRIYv.png" width="18"/></span>
<span class="show-link" id="show-director">Show <img alt="Show" class="absmiddle" height="16" src="https://m.media-amazon.com/images/S/sash/OPcpIfkmvSt2$q1.png" width="18"/></span>
<a name="director">Director</a> (129 credits)
</div>
<div class="filmo-category-section">
<div class="filmo-row odd" id="director-tt12089010">
<span class="year_column">
 2018
</span>
<b><a href="/title/tt12089010/">Spot of the 22nd Ji.hlava IDFF</a></b>
(Short)
<br/>
</div>
<div class="filmo-row even" id="director-tt5749596">
<span class="year_column">
 2018
</span>
<b><a href="/title/tt5749596/">The Image Book</a></b>
<br/>
</div>
<div class="filmo-row odd" id="director-tt8362750">
<span class="year_co

In [69]:
filmography_hlinks = filmography.find_all('a')
print(len(filmography_hlinks))
filmography_hlinks

753


[<a name="director">Director</a>,
 <a href="/title/tt12089010/">Spot of the 22nd Ji.hlava IDFF</a>,
 <a href="/title/tt5749596/">The Image Book</a>,
 <a href="/title/tt8362750/">Bande-annonce de 'Le livre d'image'</a>,
 <a href="/title/tt4529452/">Remerciements de Jean-Luc Godard à son Prix d'honneur du cinéma suisse</a>,
 <a href="/title/tt3495794/">Ponts de Sarajevo</a>,
 <a href="/title/tt2400275/">Goodbye to Language</a>,
 <a href="/title/tt3749888/">Khan Khanne</a>,
 <a href="/title/tt2693702/">3x3D</a>,
 <a href="/title/tt1438535/">Film socialisme</a>,
 <a href="/title/tt1712583/">Tribute to Éric Rohmer</a>,
 <a href="/title/tt1350630/">Une catastrophe</a>,
 <a href="/title/tt3307300/">Journal des Réalisateurs de Jean-Luc Godard</a>,
 <a href="/title/tt3301832/">Ecce Homo</a>,
 <a href="/title/tt2000378/">Four Short Films</a>,
 <a href="/title/tt2224421/">Reportage amateur Maquette expo</a>,
 <a href="/title/tt0932675/">Vrai faux passeport</a>,
 <a href="/title/tt3297780/">Prière

**QUESTIONS**

> Select the first link on the page.

> Now select the LAST link on the page.  Can you get the text and the URL associated with this link?

Remember `find` gets only one match, but `find_all` retrieves all matches in a list.

In [25]:
for link in soup.find_all('a')[:5]:
    print(link, '\n')

<a class="a-link-normal" href="/?ref_=bo_nb_tt_mojologo"></a> 

<a class="a-popover-trigger a-declarative" href="javascript:void(0)"><span class="mobile-menu-popover-trigger"></span>
<i class="a-icon a-icon-popover"></i></a> 

<a class="a-link-normal mojo-rollover-image" href="https://pro.imdb.com/login/ap?u=%2Flogin%2Flwa&amp;imdbPageAction=signUp&amp;ref_=mojo_nb_tt_rollover&amp;rf=mojo_nb_tt_rollover"><img alt="" src="https://m.media-amazon.com/images/G/01/IMDbPro/images/MI_121919_NYNY2020_Engagement_rollover_2x._CB1578336809_.png"/></a> 

<a class="a-popover-trigger a-declarative" href="javascript:void(0)"><span class="imdb-pro-logo"></span><i class="a-icon a-icon-popover"></i></a> 

<a class="a-link-normal mojo-fb-logo" href="https://www.facebook.com/BoxOfficeMojo/" rel="noopener" target="_blank"></a> 



And you can match only those with a specific `id` or `class` if you'd like.  Here are all the elements labeled with the "mojo-navigtaion-tab" class.

In [26]:
for element in soup.find_all(class_='mojo-navigation-tab'):
    print(element.prettify())

<a class="a-size-medium a-link-normal mojo-navigation-tab" href="/date/?ref_=bo_nb_tt_tab">
 Domestic
</a>

<a class="a-size-medium a-link-normal mojo-navigation-tab" href="/intl/?ref_=bo_nb_tt_tab">
 International
</a>

<a class="a-size-medium a-link-normal mojo-navigation-tab" href="/year/world/?ref_=bo_nb_tt_tab">
 Worldwide
</a>

<a class="a-size-medium a-link-normal mojo-navigation-tab" href="/calendar/?ref_=bo_nb_tt_tab">
 Calendar
</a>

<a class="a-size-medium a-link-normal mojo-navigation-tab" href="/charts/overall/?ref_=bo_nb_tt_tab">
 All Time
</a>

<a class="a-size-medium a-link-normal mojo-navigation-tab" href="/showdown/?ref_=bo_nb_tt_tab">
 Showdowns
</a>

<a class="a-size-medium a-link-normal mojo-navigation-tab" href="/brand/?ref_=bo_nb_tt_tab">
 Indices
</a>
<a class="a-size-base a-link-normal mojo-navigation-tab mojo-navigation-tab-active" href="/title/tt0118715/?ref_=bo_tt_tab#tabs">
 Releases
</a>

<a class="a-size-base a-link-normal mojo-navigation-tab" href="/titl

It's important to remember `find` and `find_all` return BeautifulSoup elements. You can continue searching these elements, thus chaining commands together.

Basic earnings information can be found in the `div` with the "mojo-performance-summary-table" class.  Let's extract the domestic gross from this element.

<br>
<img src="images/biglebow_table.png" alt="Big Lebowski Table" style="width: 500px;"/>

In [27]:
print(soup.find(class_='mojo-performance-summary-table').prettify())

<div class="a-section a-spacing-none mojo-performance-summary-table">
 <h2 class="a-size-large a-text-bold">
  All Releases
 </h2>
 <div class="a-section a-spacing-none">
  <span class="a-size-small">
   Domestic (
   <span class="percent">
    38.6%
   </span>
   )
  </span>
  <br/>
  <span class="a-size-medium a-text-bold">
   <span class="money">
    $18,034,458
   </span>
  </span>
 </div>
 <div class="a-section a-spacing-none">
  <span class="a-size-small">
   International (
   <span class="percent">
    61.4%
   </span>
   )
  </span>
  <br/>
  <span class="a-size-medium a-text-bold">
   <span class="money">
    $28,701,011
   </span>
  </span>
 </div>
 <div class="a-section a-spacing-none">
  <span class="a-size-small">
   Worldwide
  </span>
  <br/>
  <span class="a-size-medium a-text-bold">
   <span class="money">
    $46,735,469
   </span>
  </span>
 </div>
</div>



In [28]:
soup.find(class_='mojo-performance-summary-table').find_all('span', class_='money')

[<span class="money">$18,034,458</span>,
 <span class="money">$28,701,011</span>,
 <span class="money">$46,735,469</span>]

Text needs to be extracted from one element at a time.  To get the domestic gross:

In [29]:
soup.find(class_='mojo-performance-summary-table').find_all('span', class_='money')[0].text

'$18,034,458'

You can also find using an `id`; remember id should be unique to just one element.

In [30]:
print(soup.find(id='tabs').prettify())

<div class="mojo-link-bar-internal mojo-flex mojo-flex-h" id="tabs">
 <div class="mojo-tab-container">
  <a class="a-size-base a-link-normal mojo-navigation-tab mojo-navigation-tab-active" href="/title/tt0118715/?ref_=bo_tt_tab#tabs">
   Releases
  </a>
  <a class="a-size-base a-link-normal mojo-navigation-tab" href="/title/tt0118715/credits/?ref_=bo_tt_tab#tabs">
   Cast and Crew
  </a>
  <div class="a-section mojo-disabled-tab">
   All-Time Rankings
  </div>
  <a class="a-size-base a-link-normal mojo-navigation-tab" href="/title/tt0118715/news/?ref_=bo_tt_tab#tabs">
   Related Stories
  </a>
  <a class="a-size-base a-link-normal mojo-navigation-tab" href="/title/tt0118715/similar/?ref_=bo_tt_tab#tabs">
   Similar Movies
  </a>
 </div>
</div>



### Web Scraping Pipeline

Now that we have the basics, let's practice web scraping.  **The main goal of web scraping is to extract data by taking advantage of a site's consistent format.**  That is, the code you write for one page on a website can hopefully be used on multiple pages to gather more information automatically.

Let's create code to get the following information for the movies on Box Office Mojo:
- Movie title
- Domestic gross
- Runtime
- MPAA rating
- Release date

#### Movie Title

In [31]:
soup.find('title')

<title dir="ltr">The Big Lebowski - Box Office Mojo</title>

In [32]:
title_string = soup.find('title').text

title_string

'The Big Lebowski - Box Office Mojo'

In [33]:
title_string.split('-')

['The Big Lebowski ', ' Box Office Mojo']

In [34]:
title = title_string.split('-')[0].strip()

title

'The Big Lebowski'

#### Domestic Gross: 

As we saw previously, the domestic gross can be found in a `span` within the "mojo-performance-summary-table" `div`.

In [35]:
dtg = soup.find(class_='mojo-performance-summary-table').find_all('span', class_='money')[0].text
dtg

'$18,034,458'

The remainder of the information lives in this neighboring `div`.

<img src="images/biglebow_info.png" alt="Big Lebowski Information" style="width: 500px;"/>

#### Runtime: `.findNext()`

Sometimes you can find the information you are looking for by using text matching.  But note this must be an exact match!

In [36]:
soup.find(text='Run')  #does not match

In [37]:
soup.find(text='Running Time')  

'Running Time'

Alternatively, we could use [regular expressions](https://docs.python.org/3/library/re.html).

In [38]:
import re
runtime_regex = re.compile('Run')
soup.find(text=runtime_regex)

'Running Time'

In [39]:
rt_string = soup.find(text=re.compile('Run'))
print(rt_string)

Running Time


In [40]:
type(rt_string)

bs4.element.NavigableString

The string we found is still a Beautiful Soup element. This means we can use it to navigate to the next element in the HTML, which is a `span` containing the actual runtime.

In [41]:
rt_string.findNext()

<span>1 hr 57 min</span>

The `.findNext()` method can be incredibly useful when the information you want to find doesn't have a obvious tag, class, id, etc.

Let's clean this value up into usable data.

In [42]:
rt = rt_string.findNext().text
rt = rt.split()
minutes = int(rt[0])*60 + int(rt[2])
print(minutes)

117


#### MPAA Rating, Release Date

_**STEP 1:** Create function to grab values_ 

The text matching method can also help us get runtime, rating, and release date, so let's make a reuable function.

In [43]:
def get_movie_value(soup, field_name):
    
    '''Grab a value from Box Office Mojo HTML
    
    Takes a string attribute of a movie on the page and returns the string in
    the next sibling object (the value for that attribute) or None if nothing is found.
    '''
    
    obj = soup.find(text=re.compile(field_name))
    
    if not obj: 
        return None
    
    # this works for most of the values
    next_element = obj.findNext()
    
    if next_element:
        return next_element.text 
    else:
        return None

In [44]:
# runtime
runtime = get_movie_value(soup,'Run')
print(runtime)

1 hr 57 min


In [45]:
# rating
rating = get_movie_value(soup,'MPAA')
print(rating)

R


In [46]:
release_date = get_movie_value(soup,'Release Date')
print(release_date)

March 6, 1998
            (Domestic)


In [47]:
release_date = release_date.split('\n')[0]  #Select the only the date
print(release_date)

March 6, 1998


_**STEP 2:** Create helper functions to parse strings into appropriate data types_

The returned values all need a bit of formatting before we can work with this data.  Here are a few helper functions.

In [48]:
import dateutil.parser

def money_to_int(moneystring):
    moneystring = moneystring.replace('$', '').replace(',', '')
    return int(moneystring)

def runtime_to_minutes(runtimestring):
    runtime = runtimestring.split()
    try:
        minutes = int(runtime[0])*60 + int(runtime[2])
        return minutes
    except:
        return None

def to_date(datestring):
    date = dateutil.parser.parse(datestring)
    return date

_**STEP 3:** Apply these conversions_

Let's get these values again and format them all in one swoop. (Note: Rating is already correct as a string.)

In [49]:
raw_domestic_total_gross = dtg
domestic_total_gross = money_to_int(raw_domestic_total_gross)

raw_runtime = get_movie_value(soup,'Running')
runtime = runtime_to_minutes(raw_runtime)

raw_release_date = get_movie_value(soup,'Release Date').split('\n')[0]
release_date = to_date(raw_release_date)

#### Put Results in Dictionary

Now that we have results for all five quantities, we can store them in a dictionary.

In [50]:
headers = ['movie title', 'domestic total gross',
           'runtime (mins)', 'rating', 'release date']

movie_data = []
movie_dict = dict(zip(headers, [title,
                                domestic_total_gross,
                                runtime,
                                rating, 
                                release_date]))

movie_data.append(movie_dict)
movie_data

[{'movie title': 'The Big Lebowski',
  'domestic total gross': 18034458,
  'runtime (mins)': 117,
  'rating': 'R',
  'release date': datetime.datetime(1998, 3, 6, 0, 0)}]

**QUESTION**

> Why might we want to store these data in a dictionary?  Why did we put the dictionary in a list?

### Scraping Tables

Let's take a look at the [top G-rated movies](https://www.boxofficemojo.com/chart/mpaa_title_lifetime_gross/?by_mpaa=G) of Box Office Mojo.  How could we pull all the data from this main page?

First request the HTML and parse it with Beautiful Soup.

In [51]:
url = 'https://www.boxofficemojo.com/chart/mpaa_title_lifetime_gross/?by_mpaa=G'

response = requests.get(url)
page = response.text

soup = BeautifulSoup(page,"lxml")

Now find the main table; its the only `table` on the page.

In [52]:
table = soup.find('table')
table

<table class="a-bordered a-horizontal-stripes a-size-base a-span12 mojo-body-table mojo-table-annotated"><tr><th class="a-text-left mojo-field-type-title a-nowrap"><span title="Title">Title</span>
</th><th class="a-text-right mojo-field-type-rank a-nowrap"><span title="Rank">Rank</span>
</th><th class="a-text-right mojo-field-type-money a-nowrap"><span title="Lifetime Gross">Lifetime Gross</span>
</th><th class="a-text-right mojo-field-type-rank a-nowrap"><span title="Overall Rank">Overall Rank</span>
</th><th class="a-text-left mojo-field-type-year a-nowrap"><span title="Year">Year</span>
</th></tr><tr><td class="a-text-left mojo-header-column mojo-truncate mojo-field-type-title"><a class="a-link-normal" href="/title/tt1979376/?ref_=bo_cso_table_1">Toy Story 4</a></td><td class="a-text-right mojo-field-type-rank">1</td><td class="a-text-right mojo-field-type-money">$434,038,008</td><td class="a-text-right mojo-field-type-rank">24</td><td class="a-text-left mojo-field-type-year"><a cla

In [53]:
rows = [row for row in table.find_all('tr')]  # tr tag is for rows

Each row contains the information we want but requires more parsing.

In [54]:
rows[1]

<tr><td class="a-text-left mojo-header-column mojo-truncate mojo-field-type-title"><a class="a-link-normal" href="/title/tt1979376/?ref_=bo_cso_table_1">Toy Story 4</a></td><td class="a-text-right mojo-field-type-rank">1</td><td class="a-text-right mojo-field-type-money">$434,038,008</td><td class="a-text-right mojo-field-type-rank">24</td><td class="a-text-left mojo-field-type-year"><a class="a-link-normal" href="/year/2019/?ref_=bo_cso_table_1">2019</a></td></tr>

Remember: you can chain methods together to look for information!

In [55]:
rows[1].find_all('td')[0].find('a')['href']

'/title/tt1979376/?ref_=bo_cso_table_1'

Now grab data for the first 5 movies with a loop.

In [56]:
movies = {}

for row in rows[1:6]:
    items = row.find_all('td')
    link = items[0].find('a')
    title, url = link.text, link['href']
    movies[title] = [url] + [i.text for i in items]
    
movies

{'Toy Story 4': ['/title/tt1979376/?ref_=bo_cso_table_1',
  'Toy Story 4',
  '1',
  '$434,038,008',
  '24',
  '2019'],
 'The Lion King': ['/title/tt0110357/?ref_=bo_cso_table_2',
  'The Lion King',
  '2',
  '$422,783,777',
  '28',
  '1994'],
 'Toy Story 3': ['/title/tt0435761/?ref_=bo_cso_table_3',
  'Toy Story 3',
  '3',
  '$415,004,880',
  '30',
  '2010'],
 'Finding Nemo': ['/title/tt0266543/?ref_=bo_cso_table_4',
  'Finding Nemo',
  '4',
  '$380,843,261',
  '43',
  '2003'],
 'Monsters, Inc.': ['/title/tt0198781/?ref_=bo_cso_table_5',
  'Monsters, Inc.',
  '5',
  '$289,916,256',
  '101',
  '2001']}

### Scraping Multiple Pages

Now that we have the links for several G-rated movies we can visit each link to extract even more information about each movie.  Let's use `pandas` to help.

In [57]:
import pandas as pd

In [58]:
g_movies = pd.DataFrame(movies).T  #transpose
g_movies.columns = ['link_stub', 'title', 'rank_g_movies', 
                    'lifetime_gross', 'rank_overall', 'year']

g_movies.head()

,link_stub,title,rank_g_movies,lifetime_gross,rank_overall,year
Toy Story 4,/title/tt1979376/?ref_=bo_cso_table_1,Toy Story 4,1,"$434,038,008",24,2019
The Lion King,/title/tt0110357/?ref_=bo_cso_table_2,The Lion King,2,"$422,783,777",28,1994
Toy Story 3,/title/tt0435761/?ref_=bo_cso_table_3,Toy Story 3,3,"$415,004,880",30,2010
Finding Nemo,/title/tt0266543/?ref_=bo_cso_table_4,Finding Nemo,4,"$380,843,261",43,2003
"Monsters, Inc.",/title/tt0198781/?ref_=bo_cso_table_5,"Monsters, Inc.",5,"$289,916,256",101,2001


We'll also combine all previous steps into one helper function.

In [59]:
def get_movie_dict(link):
    '''
    From BoxOfficeMojo link stub, request movie html, parse with BeautifulSoup, and
    collect 
        - title 
        - domestic gross
        - runtime 
        - MPAA rating
        - full release date
    Return information as a dictionary.
    '''
    
    base_url = 'https://www.boxofficemojo.com'
    
    #Create full url to scrape
    url = base_url + link
    
    #Request HTML and parse
    response = requests.get(url)
    page = response.text
    soup = BeautifulSoup(page,"lxml")

    
    headers = ['movie_title', 'domestic_total_gross',
               'runtime_minutes', 'rating', 'release_date']
    
    #Get title
    title_string = soup.find('title').text
    title = title_string.split('-')[0].strip()

    #Get domestic gross
    raw_domestic_total_gross = (soup.find(class_='mojo-performance-summary-table')
                                    .find_all('span', class_='money')[0]
                                    .text
                               )
    domestic_total_gross = money_to_int(raw_domestic_total_gross)

    #Get runtime
    raw_runtime = get_movie_value(soup,'Running')
    runtime = runtime_to_minutes(raw_runtime)
    
    #Get rating
    rating = get_movie_value(soup,'MPAA')

    #Get release date
    raw_release_date = get_movie_value(soup,'Release Date').split('\n')[0]
    release_date = to_date(raw_release_date)
    
    #Create movie dictionary and return
    movie_dict = dict(zip(headers, [title,
                                domestic_total_gross,
                                runtime,
                                rating, 
                                release_date]))

    return movie_dict

Now we just need to pass each link stub to this function.

In [60]:
g_movies_page_info_list = []

for link in g_movies.link_stub:
    g_movies_page_info_list.append(get_movie_dict(link))

In [61]:
g_movies_page_info_list

[{'movie_title': 'Toy Story 4',
  'domestic_total_gross': 434038008,
  'runtime_minutes': 100,
  'rating': 'G',
  'release_date': datetime.datetime(2019, 6, 20, 0, 0)},
 {'movie_title': 'The Lion King',
  'domestic_total_gross': 422783777,
  'runtime_minutes': 88,
  'rating': 'G',
  'release_date': datetime.datetime(1994, 6, 15, 0, 0)},
 {'movie_title': 'Toy Story 3',
  'domestic_total_gross': 415004880,
  'runtime_minutes': 103,
  'rating': None,
  'release_date': datetime.datetime(2010, 6, 16, 0, 0)},
 {'movie_title': 'Finding Nemo',
  'domestic_total_gross': 380843261,
  'runtime_minutes': 100,
  'rating': None,
  'release_date': datetime.datetime(2003, 5, 30, 0, 0)},
 {'movie_title': 'Monsters, Inc.',
  'domestic_total_gross': 289916256,
  'runtime_minutes': 92,
  'rating': 'G',
  'release_date': datetime.datetime(2001, 11, 2, 0, 0)}]

In [62]:
g_movies_page_info = pd.DataFrame(g_movies_page_info_list)  #convert list of dict to df
g_movies_page_info.set_index('movie_title', inplace=True)

g_movies_page_info

,domestic_total_gross,rating,release_date,runtime_minutes
movie_title,,,,
Toy Story 4,434038008,G,2019-06-20,100
The Lion King,422783777,G,1994-06-15,88
Toy Story 3,415004880,None,2010-06-16,103
Finding Nemo,380843261,None,2003-05-30,100
"Monsters, Inc.",289916256,G,2001-11-02,92


(Note: the rating is indeed missing from a few of these pages!  How could you fix that?)

We can now match this back up with the movie information collected from the table by merging these dataframes.

In [63]:
g_movies = g_movies.merge(g_movies_page_info, left_index=True, right_index=True)

g_movies

,link_stub,title,rank_g_movies,lifetime_gross,rank_overall,year,domestic_total_gross,rating,release_date,runtime_minutes
Toy Story 4,/title/tt1979376/?ref_=bo_cso_table_1,Toy Story 4,1,"$434,038,008",24,2019,434038008,G,2019-06-20,100
The Lion King,/title/tt0110357/?ref_=bo_cso_table_2,The Lion King,2,"$422,783,777",28,1994,422783777,G,1994-06-15,88
Toy Story 3,/title/tt0435761/?ref_=bo_cso_table_3,Toy Story 3,3,"$415,004,880",30,2010,415004880,None,2010-06-16,103
Finding Nemo,/title/tt0266543/?ref_=bo_cso_table_4,Finding Nemo,4,"$380,843,261",43,2003,380843261,None,2003-05-30,100
"Monsters, Inc.",/title/tt0198781/?ref_=bo_cso_table_5,"Monsters, Inc.",5,"$289,916,256",101,2001,289916256,G,2001-11-02,92


## Recap

- Beautiful Soup is a powerful HTML parser
- You can locate one element with `.find()` or all matching elements with `.find_all()`
- To select specific elements, you can filter by tags like `class` or `id` 
- You can also find items using text matching and `.findNext()`, `.findNextSibling()`, `.findChild()`, etc.
- Once you know how to scrape one page, you can scale up by systematically visiting other similar pages.

### Limitations
Beautiful Soup has its limitations though.  For example, we can't use Beautiful Soup if a page:
- Requires us to input a password
- Reveals information we want only when we interact with it
- Generates dynamically (with JavaScript) rather than statically serving HTML

For these situations we need a different tool, like **Selenium** -- coming soon!